# Clustering Crypto

In [1]:
#!pip install -U altair

     |████████████████████████████████| 727 kB 9.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [1]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
#import hvplot.pandas
import altair as alt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import json
from path import Path

### Fetching Cryptocurrency Data

In [2]:
"""# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
response = requests.get(url).json()
"""

'# Use the following endpoint to fetch json data\nurl = "https://min-api.cryptocompare.com/data/all/coinlist"\nresponse = requests.get(url).json()\n'

In [3]:
"""# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_df = pd.DataFrame(coin for coin in response['Data'].values())
crypto_df.head(5)"""

"# Create a DataFrame \n# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.\ncrypto_df = pd.DataFrame(coin for coin in response['Data'].values())\ncrypto_df.head(5)"

In [4]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [5]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df =crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [6]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [7]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'].notnull()]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [8]:
# Remove the 'IsTrading' column
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [9]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna().copy()
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [11]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.where(crypto_df.values != 'N/A').dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
coins_name

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=['CoinName'])
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [14]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Standardize data

X = StandardScaler().fit_transform(X)



### Reducing Dimensions Using PCA

In [16]:
# Use PCA to reduce dimensions to 3 principal components
# Initialize PCA model
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)

In [17]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"]
).set_index(crypto_df.index)
pcs_df.head()


,PC 1,PC 2,PC 3
42,-0.335805,1.028750,-0.512205
404,-0.319125,1.028975,-0.512844
1337,2.317185,1.721047,-0.679605
BTC,-0.141737,-1.286906,0.164394
ETH,-0.140473,-2.022636,0.325899


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [18]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
#df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")
alt.Chart(df_elbow).mark_line(point=True).encode(
    alt.X('k', scale=alt.Scale(zero=False)),
    alt.Y('inertia', scale=alt.Scale(zero=False))
    ).properties(
        title='Elbow Curve'
    )



alt.Chart(...)

Running K-Means with `k=4`

In [19]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([crypto_df, pcs_df, coins_name], axis=1)
clustered_df["Class"] = model.labels_

In [20]:
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335805,1.028750,-0.512205,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319125,1.028975,-0.512844,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.317185,1.721047,-0.679605,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141737,-1.286906,0.164394,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.140473,-2.022636,0.325899,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.166777,-1.145124,-0.000491,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.404526,1.241046,-0.407568,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.147368,-2.187945,0.351917,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.138912,-2.022728,0.325860,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.134903,-2.037424,0.452071,ZCash,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [21]:
# Create a 3D-Scatter with the PCA data and the clusters
#fig = px.scatter_3d(
#    clustered_df,
#    hover_name="CoinName",
#    hover_data=["Algorithm"],
#    x="PC 3",
#    y="PC 2",
#    z="PC 1",
#    color="Class",
#    symbol="Class",
#    width=800,
#)
#fig.update_layout(legend=dict(x=0, y=1))
#fig.show()

alt.Chart(clustered_df).mark_point().encode(
    alt.X('PC 1', scale=alt.Scale(zero=False)),
    alt.Y('PC 2', scale=alt.Scale(zero=False)),
    color=alt.Color('Class', scale=alt.Scale(scheme='dark2')),
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()


alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [28]:
# Table with tradable cryptos
#clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)
display(clustered_df)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000e+00,4.200000e-11,-0.335805,1.028750,-0.512205,42 Coin,1
404,Scrypt,PoW/PoS,1.065855e-03,5.320000e-04,-0.319125,1.028975,-0.512844,404Coin,1
1337,X13,PoW/PoS,2.957551e-02,3.141593e-01,2.317185,1.721047,-0.679605,EliteCoin,1
BTC,SHA-256,PoW,1.810842e-05,2.100000e-05,-0.141737,-1.286906,0.164394,Bitcoin,0
ETH,Ethash,PoW,1.087731e-04,0.000000e+00,-0.140473,-2.022636,0.325899,Ethereum,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.020225e-03,2.000000e-03,2.462163,0.790206,-0.238908,ZEPHYR,1
GAP,Scrypt,PoW/PoS,1.508199e-05,2.500000e-04,-0.333848,1.028645,-0.512255,Gapcoin,1
BDX,CryptoNight,PoW,9.901351e-04,1.400223e-03,0.327503,-2.302579,0.343417,Beldex,0
ZEN,Equihash,PoW,7.370282e-06,2.100000e-05,-0.134904,-2.037424,0.452071,Horizen,0


In [24]:
# Print the total number of tradable cryptocurrencies
len(clustered_df)

532

#### Scatter Plot with Tradable Cryptocurrencies

In [25]:
# Scale data to create the scatter plot
#scaler = MinMaxScaler()
#scaler.fit(clustered_df)
clustered_df[['TotalCoinsMined', 'TotalCoinSupply']] = MinMaxScaler().fit_transform(clustered_df[['TotalCoinsMined', 'TotalCoinSupply']])

In [26]:
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000e+00,4.200000e-11,-0.335805,1.028750,-0.512205,42 Coin,1
404,Scrypt,PoW/PoS,1.065855e-03,5.320000e-04,-0.319125,1.028975,-0.512844,404Coin,1
1337,X13,PoW/PoS,2.957551e-02,3.141593e-01,2.317185,1.721047,-0.679605,EliteCoin,1
BTC,SHA-256,PoW,1.810842e-05,2.100000e-05,-0.141737,-1.286906,0.164394,Bitcoin,0
ETH,Ethash,PoW,1.087731e-04,0.000000e+00,-0.140473,-2.022636,0.325899,Ethereum,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.020225e-03,2.000000e-03,2.462163,0.790206,-0.238908,ZEPHYR,1
GAP,Scrypt,PoW/PoS,1.508199e-05,2.500000e-04,-0.333848,1.028645,-0.512255,Gapcoin,1
BDX,CryptoNight,PoW,9.901351e-04,1.400223e-03,0.327503,-2.302579,0.343417,Beldex,0
ZEN,Equihash,PoW,7.370282e-06,2.100000e-05,-0.134904,-2.037424,0.452071,Horizen,0


In [27]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

#clustered_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', hover_cols=['CoinName'], 
#                  legend='top', height=400, width=400)

alt.Chart(clustered_df).mark_point().encode(
    alt.X('TotalCoinsMined', scale=alt.Scale(zero=False)),
    alt.Y('TotalCoinSupply', scale=alt.Scale(zero=False)),
    color=alt.Color('Class', scale=alt.Scale(scheme='dark2')),
    tooltip=["CoinName"]
).interactive()

alt.Chart(...)